In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [1]:
# STEP 1 — Mount Drive and set up paths (final)

# Install libs first (quiet)
!pip install -q keras-tcn

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os
from pathlib import Path
import numpy as np
import random

# ---- Base folders
BASE = Path("/content/drive") / "MyDrive" / "GoldForecasting_Project (ISY5002)"
DATA = BASE / "Data"
MODELS = BASE / "Models"
MODELS_PROPHET = MODELS / "Prophet"
MODELS_TCN = MODELS / "TCN"
OUTPUTS = BASE / "Outputs"

# Create folders if needed
for p in [DATA, MODELS, MODELS_PROPHET, MODELS_TCN, OUTPUTS]:
    p.mkdir(parents=True, exist_ok=True)

# ---- Canonical file paths (match your screenshots)
# Prophet
PROPHET_JOBLIB = MODELS_PROPHET / "prophet_model.joblib"
PROPHET_OUTCSV = MODELS_PROPHET / "prophet_output.csv"
PROPHET_REQ = MODELS_PROPHET / "requirements_prophet.txt"

# TCN v1 (baseline)
TCN1_KERAS = MODELS_TCN / "tcn_v1_baseline.keras"
TCN1_JOBLIB = MODELS_TCN / "tcn_v1_baseline.joblib"
TCN1_OUTCSV = MODELS_TCN / "tcn_v1_baseline_output.csv"
TCN1_REQ = MODELS_TCN / "requirements_tcn_v1_baseline.txt"

# TCN v2 (improved)
TCN2_KERAS = MODELS_TCN / "tcn_v2_improved.keras"
TCN2_JOBLIB = MODELS_TCN / "tcn_v2_improved.joblib"
TCN2_OUTCSV = MODELS_TCN / "tcn_v2_improved_output.csv"
TCN2_REQ = MODELS_TCN / "requirements_tcn_v2.txt"

# Quick verification
print("Drive OK:", BASE.exists())
print("Prophet dir:", MODELS_PROPHET)
print("TCN dir:", MODELS_TCN)
for p in [PROPHET_JOBLIB, TCN1_KERAS, TCN2_KERAS]:
    print("Sample path:", p)

# (Optional) global seed for later reproducibility
GLOBAL_SEED = 99
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
try:
    import tensorflow as tf
    tf.random.set_seed(GLOBAL_SEED)
    print("Seeds set.")
except Exception as e:
    print("TensorFlow not loaded yet (will set later).", e)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive OK: True
Prophet dir: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/Prophet
TCN dir: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/TCN
Sample path: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/Prophet/prophet_model.joblib
Sample path: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/TCN/tcn_v1_baseline.keras
Sample path: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/TCN/tcn_v2_improved.keras
Seeds set.


In [8]:
# ================================================================
# STEP 2 – Load & Prepare Gold Dataset (Auto-download if missing)
# ================================================================

import os
import pandas as pd
import numpy as np
import random
import yfinance as yf

# Reuse the global seed for reproducibility
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)

# Define dataset file path
file_path = os.path.join(DATA, "XAU_USD_Spot_2025.csv")

# Check if file exists – if not, auto-download from Yahoo Finance
if not os.path.exists(file_path):
    print("📥 Local gold data not found. Auto-downloading from Yahoo Finance...")

    try:
        # Download COMEX gold futures (GC=F)
        df = yf.download("GC=F", start="2021-01-01", end="2025-12-31", interval="1d")

        # Reset index and rename columns for consistency
        df.reset_index(inplace=True)
        df.rename(columns={
            "Date": "Date",
            "Open": "Open",
            "High": "High",
            "Low": "Low",
            "Close": "Close",
            "Adj Close": "Adj_Close",
            "Volume": "Vol"
        }, inplace=True)

        # Drop NaN rows if any
        df.dropna(subset=["Close"], inplace=True)

        # Add daily percentage change column
        df["Change_%"] = df["Close"].pct_change() * 100
        df.dropna(inplace=True)

        # Save to Drive for reuse
        df.to_csv(file_path, index=False)
        print(f"✅ Gold dataset downloaded and saved to:\n{file_path}")

    except Exception as e:
        print("❌ Yahoo Finance download failed:", e)
        raise

else:
    # Load from Drive if already saved
    df = pd.read_csv(file_path)
    print(f"📂 Loaded existing dataset from:\n{file_path}")

# Final data preparation
df["Date"] = pd.to_datetime(df["Date"])
df.sort_values("Date", inplace=True)

# Dataset summary
print(f"✅ Final dataset ready: {len(df)} rows, "
      f"{df['Date'].min().date()} → {df['Date'].max().date()}")

# Quick preview
df.head()


📂 Loaded existing dataset from:
/content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Data/XAU_USD_Spot_2025.csv
✅ Final dataset ready: 1208 rows, 2021-01-05 → 2025-10-22


,Date,Close,High,Low,Open,Vol,Change_%
1,2021-01-05,1952.699951171875,1952.699951171875,1941.300048828125,1941.699951171875,113,0.411375
2,2021-01-06,1906.9000244140625,1959.9000244140625,1901.5,1952.0,331,-2.345467
3,2021-01-07,1912.300048828125,1926.699951171875,1912.0,1922.5999755859375,122,0.283183
4,2021-01-08,1834.0999755859375,1908.0,1834.0999755859375,1908.0,60,-4.089320
5,2021-01-11,1849.5999755859375,1849.5999755859375,1826.5,1826.5,20,0.845101


In [9]:
# ================================================================
# STEP 3 – Load Latest Trained Models (26-Oct-2025 builds)
# ================================================================

import joblib
from tensorflow.keras.models import load_model
from tcn import TCN

# Prophet model (latest)
prophet_model_path = os.path.join(MODELS_PROPHET, "prophet_model.joblib")
m_prophet = joblib.load(prophet_model_path)
print(f"✅ Prophet model loaded from: {prophet_model_path}")

# TCN models (latest)
tcn_v1_model_path = os.path.join(MODELS_TCN, "tcn_v1_baseline.keras")
tcn_v2_model_path = os.path.join(MODELS_TCN, "tcn_v2_improved.keras")

tcn_v1 = load_model(tcn_v1_model_path, custom_objects={"TCN": TCN}, compile=False)
tcn_v2 = load_model(tcn_v2_model_path, custom_objects={"TCN": TCN}, compile=False)

print("✅ TCN v1 and v2 models loaded successfully.")

# Optional verification (check file modified dates)
import os, datetime
for path in [prophet_model_path, tcn_v1_model_path, tcn_v2_model_path]:
    t = os.path.getmtime(path)
    print(f"📂 {os.path.basename(path)} last modified: {datetime.datetime.fromtimestamp(t)}")


✅ Prophet model loaded from: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/Prophet/prophet_model.joblib
✅ TCN v1 and v2 models loaded successfully.
📂 prophet_model.joblib last modified: 2025-10-25 17:32:26
📂 tcn_v1_baseline.keras last modified: 2025-10-26 01:49:15
📂 tcn_v2_improved.keras last modified: 2025-10-26 02:33:06


In [10]:
# ================================================================
# STEP 4 – Prepare Input Data for Forecasting
# ================================================================

import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Prophet requires columns ['ds', 'y']
prophet_df = df.rename(columns={"Date": "ds", "Close": "y"})[["ds", "y"]]
print("✅ Prophet data prepared (columns ds, y).")

# --- For TCN: normalize recent historical data ---
# Keep only numeric columns
numeric_cols = ["Open", "High", "Low", "Close"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
df = df.dropna(subset=numeric_cols)  # drop rows with missing numeric data

# Scale the numeric features
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[numeric_cols])
print("✅ Scaling complete.")

# ------------------------------------------------------------
# Optional: Save the fitted scaler for future reuse
# ------------------------------------------------------------
tnc_scaler_path = os.path.join(MODELS_TCN, "tnc_scaler.joblib")
joblib.dump(scaler, tnc_scaler_path)
print(f"💾 Saved fitted TCN scaler to: {tnc_scaler_path}")

# Create fixed-size input sequences (e.g., 30-day window)
window_size = 30

def create_sequences(data, window=30):
    X = []
    for i in range(len(data) - window):
        X.append(data[i:i + window])
    return np.array(X)

# Take the latest window for prediction
X_latest = create_sequences(scaled, window_size)[-1:]
print(f"✅ Input data prepared for all 3 models (shape: {X_latest.shape}).")

# Optional: quick sanity check
print(f"📊 Latest window covers {window_size} days ending on {df['Date'].iloc[-1].date()}")


✅ Prophet data prepared (columns ds, y).
✅ Scaling complete.
💾 Saved fitted TCN scaler to: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/TCN/tnc_scaler.joblib
✅ Input data prepared for all 3 models (shape: (1, 30, 4)).
📊 Latest window covers 30 days ending on 2025-10-22


In [16]:
import os, datetime
t = os.path.getmtime(tnc_scaler_path)
print(f"🕒 Confirmed scaler timestamp: {datetime.datetime.fromtimestamp(t)}")


🕒 Confirmed scaler timestamp: 2025-10-26 03:48:18


In [17]:
from datetime import datetime, timezone, timedelta
print("UTC now:", datetime.now(timezone.utc))
print("Singapore time:", datetime.now(timezone(timedelta(hours=8))))


UTC now: 2025-10-26 03:59:35.630007+00:00
Singapore time: 2025-10-26 11:59:35.630185+08:00


In [11]:
# Clean non-numeric rows or bad data before Prophet
df = df[pd.to_numeric(df["Close"], errors="coerce").notnull()].copy()
df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df = df.dropna(subset=["Date", "Close"]).reset_index(drop=True)

prophet_df = df.rename(columns={"Date": "ds", "Close": "y"})[["ds", "y"]]


In [18]:
# ================================================================
# STEP 5 – Forecast Using Prophet + TCN (v1 & v2) and Summarize
# ================================================================

import numpy as np

# --- 5.1: Prepare Prophet input (last date + 1 day) ---
future_date = prophet_df["ds"].iloc[-1] + pd.Timedelta(days=1)
future_df = pd.DataFrame({"ds": [future_date]})

# Prophet prediction (USD)
prophet_forecast = m_prophet.predict(future_df)
prophet_pred = float(prophet_forecast["yhat"].values[0])
print(f"📈 Prophet predicted close for {future_date.date()}: ${prophet_pred:,.2f}")

# --- 5.2: Load or reuse shared TCN scaler ---
scaler_path = os.path.join(MODELS_TCN, "tcn_scaler.joblib")
scaler = joblib.load(scaler_path)
features = list(scaler.feature_names_in_) if hasattr(scaler, "feature_names_in_") else \
           ["Open", "High", "Low", "Close", "Change_%"]
print(f"✅ Using TCN scaler with features: {features}")

# --- 5.3: Prepare latest scaled input window ---
scaled = scaler.transform(df[features])
window_size = 30
X_latest = np.expand_dims(scaled[-window_size:], axis=0)

# --- Predict scaled outputs from both TCN models ---
tcn_v1_pred_scaled = tcn_v1.predict(X_latest)
tcn_v2_pred_scaled = tcn_v2.predict(X_latest)

# --- Helper: convert scaled Close back to USD ---
def inverse_close_to_usd(pred_scaled, scaler, features, clip=True, jitter=0.5):
    s = float(np.array(pred_scaled).reshape(-1)[0])
    if clip:
        s = float(np.clip(s, 0.0, 1.0))
    close_idx = features.index("Close")
    close_min = scaler.data_min_[close_idx]
    close_max = scaler.data_max_[close_idx]
    unscaled = close_min + s * (close_max - close_min)
    unscaled += np.random.uniform(-jitter, jitter)
    return round(unscaled, 2)

# Convert both TCN predictions to USD
tcn_v1_pred_usd = inverse_close_to_usd(tcn_v1_pred_scaled, scaler, features)
tcn_v2_pred_usd = inverse_close_to_usd(tcn_v2_pred_scaled, scaler, features)

# --- 5.4: Combine predictions (simple ensemble average) ---
ensemble_pred = round(np.mean([prophet_pred, tcn_v1_pred_usd, tcn_v2_pred_usd]), 2)

# --- 5.5: Display results ---
print("\n🔥 Final Forecast Summary:")
print(f"• Prophet (Baseline): ${prophet_pred:,.2f}")
print(f"• TCN v1 (Baseline):  ${tcn_v1_pred_usd:,.2f}")
print(f"• TCN v2 (Improved):  ${tcn_v2_pred_usd:,.2f}")
print(f"• 🧠 Ensemble Average: ${ensemble_pred:,.2f}")


📈 Prophet predicted close for 2025-10-23: $3,587.39
✅ Using TCN scaler with features: ['Open', 'High', 'Low', 'Close', 'Change_%']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

🔥 Final Forecast Summary:
• Prophet (Baseline): $3,587.39
• TCN v1 (Baseline):  $3,425.30
• TCN v2 (Improved):  $2,896.76
• 🧠 Ensemble Average: $3,303.15


In [19]:
import os
from datetime import datetime

print("📂 Checking model files...")

for path in [
    os.path.join(MODELS_PROPHET, "prophet_model.joblib"),
    os.path.join(MODELS_TCN, "tcn_v1_baseline.keras"),
    os.path.join(MODELS_TCN, "tcn_v1_baseline.joblib"),
    os.path.join(MODELS_TCN, "tcn_v2_improved.keras"),
    os.path.join(MODELS_TCN, "tcn_v2_improved.joblib"),
]:
    if os.path.exists(path):
        t = os.path.getmtime(path)
        print(f"✅ {os.path.basename(path)}  →  last modified: {datetime.fromtimestamp(t)}")
    else:
        print(f"⚠️ Missing: {os.path.basename(path)}")


📂 Checking model files...
✅ prophet_model.joblib  →  last modified: 2025-10-25 17:32:26
✅ tcn_v1_baseline.keras  →  last modified: 2025-10-26 01:49:15
✅ tcn_v1_baseline.joblib  →  last modified: 2025-10-26 01:49:15
✅ tcn_v2_improved.keras  →  last modified: 2025-10-26 02:33:06
✅ tcn_v2_improved.joblib  →  last modified: 2025-10-26 02:33:06


In [20]:
# ================================================================
# STEP 6.1 – Generate Trading Signals for Each Model
# ================================================================

def get_signal(pred, last):
    """Return BUY or SELL depending on prediction vs last close."""
    return "BUY" if pred > last else "SELL"

# Get the latest actual close value
last_close = float(df["Close"].iloc[-1])

# Compute percentage direction for clarity
def get_direction(pred, last):
    pct = (pred - last) / last * 100
    arrow = "↑" if pct > 0 else "↓"
    return f"{arrow} {pct:.2f}%"

# Collect all model results
signals = {
    "Prophet (Baseline)": {
        "pred": prophet_pred,
        "signal": get_signal(prophet_pred, last_close),
        "dir": get_direction(prophet_pred, last_close)
    },
    "TCN v1 (Baseline)": {
        "pred": tcn_v1_pred_usd,
        "signal": get_signal(tcn_v1_pred_usd, last_close),
        "dir": get_direction(tcn_v1_pred_usd, last_close)
    },
    "TCN v2 (Improved)": {
        "pred": tcn_v2_pred_usd,
        "signal": get_signal(tcn_v2_pred_usd, last_close),
        "dir": get_direction(tcn_v2_pred_usd, last_close)
    }
}

print("📊 Quick Signals:\n")
for name, s in signals.items():
    color = "\033[92m" if s["signal"] == "BUY" else "\033[91m"  # green / red
    reset = "\033[0m"
    print(f"{name}: {color}{s['signal']}{reset} "
          f"(pred {s['pred']:.2f}, last {last_close:.2f}, {s['dir']})")


📊 Quick Signals:

Prophet (Baseline): SELL (pred 3587.39, last 4146.80, ↓ -13.49%)
TCN v1 (Baseline): SELL (pred 3425.30, last 4146.80, ↓ -17.40%)
TCN v2 (Improved): SELL (pred 2896.76, last 4146.80, ↓ -30.14%)


In [21]:
# ================================================================
# STEP 6.2 – Enhanced Visual Dashboard for Model Comparison
# ================================================================

import pandas as pd
from IPython.display import display, HTML

# Create summary DataFrame
summary_df = pd.DataFrame([
    ["Prophet (Baseline)", prophet_pred, last_close, signals["Prophet (Baseline)"]["dir"], signals["Prophet (Baseline)"]["signal"]],
    ["TCN v1 (Baseline)", tcn_v1_pred_usd, last_close, signals["TCN v1 (Baseline)"]["dir"], signals["TCN v1 (Baseline)"]["signal"]],
    ["TCN v2 (Improved)", tcn_v2_pred_usd, last_close, signals["TCN v2 (Improved)"]["dir"], signals["TCN v2 (Improved)"]["signal"]]
], columns=["Model", "Predicted Price (USD)", "Last Close (USD)", "Direction", "Signal"])


# --- 🔸 Color functions for direction and signal ---
def color_signal(val):
    if val == "BUY":
        return "background-color: #c6efce; color: #006100; font-weight: bold;"  # green
    elif val == "SELL":
        return "background-color: #ffc7ce; color: #9c0006; font-weight: bold;"  # red
    return ""

def color_direction(val):
    if "↑" in val:
        return "color: #006100; font-weight: bold;"  # green arrow up
    elif "↓" in val:
        return "color: #9c0006; font-weight: bold;"  # red arrow down
    return ""


# --- 🎨 Apply style ---
styled = (
    summary_df.style
    .map(color_signal, subset=["Signal"])
    .map(color_direction, subset=["Direction"])
    .set_properties(
        subset=["Model", "Predicted Price (USD)", "Last Close (USD)"],
        **{"text-align": "center", "border": "1px solid #ccc", "color": "#222"}
    )
    .set_table_styles([
        {"selector": "th", "props": [
            ("background-color", "#1f77b4"),
            ("color", "white"),
            ("font-weight", "bold"),
            ("text-align", "center"),
            ("border-bottom", "2px solid #000")
        ]},
        {"selector": "td", "props": [("border", "1px solid #ddd"), ("padding", "6px")]}
    ])
)

print("\n✅ Final Model Comparison & Trading Dashboard")
display(styled)



✅ Final Model Comparison & Trading Dashboard


,Model,Predicted Price (USD),Last Close (USD),Direction,Signal
0,Prophet (Baseline),3587.391689,4146.799805,↓ -13.49%,SELL
1,TCN v1 (Baseline),3425.300000,4146.799805,↓ -17.40%,SELL
2,TCN v2 (Improved),2896.760000,4146.799805,↓ -30.14%,SELL


In [22]:
# ================================================================
# STEP 6.3 – Consensus Vote & Final Trading Advice
# ================================================================

# Count the votes
buy_votes = sum(s["signal"] == "BUY" for s in signals.values())
sell_votes = sum(s["signal"] == "SELL" for s in signals.values())

# Decide consensus
if buy_votes > sell_votes:
    final_signal = "BUY"
    final_direction = "↑ Uptrend Expected"
    color_bg, color_text = "#c6efce", "#006100"
elif sell_votes > buy_votes:
    final_signal = "SELL"
    final_direction = "↓ Downtrend Expected"
    color_bg, color_text = "#ffc7ce", "#9c0006"
else:
    final_signal = "NEUTRAL"
    final_direction = "→ Sideways / Wait"
    color_bg, color_text = "#fff2cc", "#7f6000"

# Append a summary row to the same DataFrame
summary_df.loc[len(summary_df)] = [
    "🧭 Consensus Decision",
    "", "",
    final_direction,
    final_signal
]

# Apply styling again (reuse same color functions)
styled = (
    summary_df.style
    .map(color_signal, subset=["Signal"])
    .map(color_direction, subset=["Direction"])
    .set_properties(
        subset=["Model", "Predicted Price (USD)", "Last Close (USD)"],
        **{"text-align": "center", "border": "1px solid #ccc", "color": "#222"}
    )
    .set_table_styles([
        {"selector": "th", "props": [
            ("background-color", "#1f77b4"),
            ("color", "white"),
            ("font-weight", "bold"),
            ("text-align", "center"),
            ("border-bottom", "2px solid #000")
        ]},
        {"selector": "td", "props": [("border", "1px solid #ddd"), ("padding", "6px")]}
    ])
)

print("\n🧭 Final Consensus Decision Summary")
display(styled)

# Optional: print a friendly message below the table
print(f"\n📢 Market Advice: Based on {buy_votes} BUY vs {sell_votes} SELL votes → "
      f"**{final_signal}** ({final_direction})\n")



🧭 Final Consensus Decision Summary


,Model,Predicted Price (USD),Last Close (USD),Direction,Signal
0,Prophet (Baseline),3587.391689,4146.799805,↓ -13.49%,SELL
1,TCN v1 (Baseline),3425.300000,4146.799805,↓ -17.40%,SELL
2,TCN v2 (Improved),2896.760000,4146.799805,↓ -30.14%,SELL
3,🧭 Consensus Decision,,,↓ Downtrend Expected,SELL



📢 Market Advice: Based on 0 BUY vs 3 SELL votes → **SELL** (↓ Downtrend Expected)



In [23]:
# ================================================================
# STEP 7 – Save Final Ensemble Metadata + requirements.txt
# ================================================================

import joblib, json, os, datetime, subprocess

# --- Ensemble metadata dictionary ---
ENSEMBLE_META = {
    "timestamp": str(datetime.datetime.now()),
    "models": {
        "Prophet": os.path.join(MODELS_PROPHET, "prophet_model.joblib"),
        "TCN_v1": os.path.join(MODELS_TCN, "tcn_v1_baseline.keras"),
        "TCN_v2": os.path.join(MODELS_TCN, "tcn_v2_improved.keras"),
        "Scaler": os.path.join(MODELS_TCN, "tnc_scaler.joblib"),
    },
    "latest_close": float(df["Close"].iloc[-1]),
    "predictions": {
        "Prophet": float(prophet_pred),
        "TCN_v1": float(tcn_v1_pred_usd),
        "TCN_v2": float(tcn_v2_pred_usd),
        "Ensemble_Avg": float(ensemble_pred),
    },
    "signals": {
        "Prophet": signals["Prophet (Baseline)"]["signal"],
        "TCN_v1": signals["TCN v1 (Baseline)"]["signal"],
        "TCN_v2": signals["TCN v2 (Improved)"]["signal"],
        "Final_Consensus": final_signal,
    },
    "direction": final_direction
}

# --- Save metadata (JSON + Joblib) ---
meta_path_json = os.path.join(MODELS_TCN, "ensemble_summary_20251026.json")
meta_path_joblib = os.path.join(MODELS_TCN, "ensemble_summary_20251026.joblib")

with open(meta_path_json, "w") as f:
    json.dump(ENSEMBLE_META, f, indent=4)
joblib.dump(ENSEMBLE_META, meta_path_joblib)

print("✅ Ensemble metadata saved successfully:")
print(f"📄 JSON : {meta_path_json}")
print(f"📦 Joblib : {meta_path_joblib}")

# ================================================================
# STEP 7.1 – Save Environment Requirements (requirements_ensemble.txt)
# ================================================================

requirements_path = os.path.join(MODELS_TCN, "requirements_ensemble.txt")

# Export installed packages into requirements file
subprocess.run(["pip", "freeze"], stdout=open(requirements_path, "w"))

print(f"🧾 Saved current environment to: {requirements_path}")


✅ Ensemble metadata saved successfully:
📄 JSON : /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/TCN/ensemble_summary_20251026.json
📦 Joblib : /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/TCN/ensemble_summary_20251026.joblib
🧾 Saved current environment to: /content/drive/MyDrive/GoldForecasting_Project (ISY5002)/Models/TCN/requirements_ensemble.txt
